In [1]:
print('here...')

here...


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import multibind as mb
import numpy as np
import pandas as pd
import torch
import bindome as bd
bd.constants.ANNOTATIONS_DIRECTORY = 'annotations'
# mb.models.MultiBind
import torch.optim as topti
import torch.utils.data as tdata
import matplotlib.pyplot as plt
import logomaker
import os
import scipy
import pickle

# Use a GPU if available, as it should be faster.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device: " + str(device))

ModuleNotFoundError: No module named 'multibind'

In [ ]:
adata = mb.bindome.datasets.scATAC.PBMCs_10x_v2(datadir='../../atac_poisson_study/data/')
peak_ids = adata.var_names

In [ ]:
adata.var['summit'] = ((adata.var['end'] + adata.var['start']) / 2).astype(int)
adata.var['summit.start'] = adata.var['summit'] - 100
adata.var['summit.end'] = adata.var['summit'] + 100

In [ ]:
adata.var['k.summit'] = adata.var['chr'] + ':' + adata.var['summit.start'].astype(str) + '-' + adata.var['summit.end'].astype(str)

In [ ]:
adata.shape

In [ ]:
n_seqs = 1000
seqs = mb.bindome.tl.get_sequences_from_bed(adata.var[['chr', 'summit.start', 'summit.end']].head(n_seqs), genome='hg38', uppercase=True)
keys = set([s[0] for s in seqs])
adata = adata[:,adata.var['k.summit'].isin(keys)]
# seqs = [[s[0], s[1].upper()] for s in seqs[0]]

In [ ]:
!which bedtools

In [ ]:
adata.shape

In [ ]:
len(seqs)

In [ ]:
# remove Ns
seqs = [[s[0], s[1].replace('N', '')] for s in seqs]
counts = adata.X.T

In [ ]:
next_data = pd.DataFrame.sparse.from_spmatrix(counts)

In [ ]:
next_data = next_data[range(100)].copy()

In [ ]:
next_data['seq'] = [s[1] for s in seqs]

In [ ]:
next_data

In [ ]:
# next_data[next_data.columns[:-1]].sum(axis=1).sort_values(ascending=False)

In [ ]:
next_data.head(1000).var(axis=1)

In [ ]:
# next_data.var(axis=1)

In [ ]:
var = []
for ri, r in next_data.iterrows():
    if ri % 10000 == 0:
        print(ri, next_data.shape)
    # print(ri, r.values[:-1], r.values[:-1].var())
    var.append(r.values[:-1].var())
    # break

In [ ]:
next_data['var'] = var

In [ ]:
# sum_index = next_data[next_data.columns[:-1]].var(axis=1).sort_values(ascending=False).index

In [ ]:
top_var = next_data[['var']].sort_values('var', ascending=False).index[:1000]

In [ ]:
# next_data = next_data.head(10000)
next_data_sel = next_data.reindex(top_var).reset_index(drop=True)

In [ ]:
next_data_sel.head()

In [ ]:
del next_data_sel['var']
next_data_sel.index = next_data_sel['seq']
del next_data_sel['seq']

In [ ]:
next_data_sel

In [ ]:
df = next_data_sel
dataset = mb.datasets.GenomicsDataset(df)
train = tdata.DataLoader(dataset=dataset, batch_size=256, shuffle=True)

In [ ]:
model_atac, best_loss = mb.tl.train_iterative(train, device, num_epochs=500, show_logo=False,
                                         early_stopping=50, log_each=50, n_kernels=2)

In [ ]:
r2 = mb.pl.R2_calculation(model_atac, train)
print("R^2:", r2)

In [ ]:
model_atac

In [ ]:
df = next_data_sel.copy()
dataset = mb.datasets.GenomicsDataset(df)
train = tdata.DataLoader(dataset=dataset, batch_size=256, shuffle=True)

In [ ]:
model_atac, best_loss = mb.tl.train_iterative(train, device, num_epochs=500, show_logo=False,
                                         early_stopping=50, log_each=50, n_kernels=2, joint_learning=True)

In [ ]:
r2 = mb.pl.R2_calculation(model_atac, train)
print("R^2:", r2)